In [219]:
# Setup
import numpy as np
import scipy as sp
import pandas as p
from IPython.display import display
from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error, r2_score

In [213]:
# Test
years = ['2011', '2012', '2013', '2014', '2015', '2016', '2017']

train_x = p.DataFrame()
train_y = p.DataFrame()
test_x = p.DataFrame()
test_y = p.DataFrame()

for year in years:
    
    # Get battings stats
    batting_df = p.read_csv(year + 'Batting.csv', 
                            skipfooter=2, 
                            engine='python')
    batting_df['Name'] = batting_df['Name'].str.split('\\').str[-1]
 
    salaries_df = p.read_csv(year + 'Salary.csv', 
                             engine='python')
    salaries_df['Name'] = salaries_df['Name'].str.split('\\').str[-1]
    
    batting_df['Pos'] = batting_df['Pos'].str[:-1]
    salaries_df['Pos'] = salaries_df['Pos'].str[:-1]
    
    # take out NAN batting averages
    batting_df = batting_df[np.isfinite(batting_df['BA'])]
    # don't care about pitchers batting
    batting_df = batting_df[batting_df.Pos != '1']
    batting_df = batting_df[~batting_df.Pos.str.startswith('/1')]
    
    batting_df = batting_df[batting_df.Name.isin(salaries_df.Name)]
    salaries_df = salaries_df[salaries_df.Name.isin(batting_df.Name)]
    salaries_df = salaries_df.sort_values('Name')
        
    if year == years[0]:
        train_x = batting_df
        train_y = salaries_df
    elif year != years[-1]:
        train_x = p.concat([train_x, batting_df])
        train_y = p.concat([train_y, salaries_df]) 
    else: 
        test_x = batting_df
        test_y = salaries_df
        
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)
train_y = train_y.fillna(0)
test_y = test_y.fillna(0)

# take out team, league
train_x.drop(train_x.columns[[0, 1, 2, 3, 4, 29]], axis=1, inplace=True)
test_x.drop(test_x.columns[[0, 1, 2, 3, 4, 29]], axis=1, inplace=True)

# get salary out of salaries
train_y = train_y.filter(['Salary'], axis=1)
test_y = test_y.filter(['Salary'], axis=1)

#drop $
train_y['Salary'] = train_y['Salary'].str.replace('$','')
test_y['Salary'] = test_y['Salary'].str.replace('$','')

display(train_x)

# check = train_x.values[95]
# print train_y.values[95]
display(train_y)

,G,PA,AB,R,H,2B,3B,HR,RBI,SB,...,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB
1,142,585,502,54,127,30,1,8,60,21,...,0.353,0.365,0.717,105.0,183,8,1,1,3,5
15,16,29,29,1,3,0,0,1,3,0,...,0.103,0.207,0.310,-18.0,6,2,0,0,0,0
16,47,98,88,9,29,4,0,5,15,0,...,0.398,0.545,0.943,153.0,48,2,0,0,0,0
18,74,262,235,18,45,9,1,4,19,1,...,0.272,0.289,0.561,58.0,68,11,2,1,0,1
23,139,511,457,63,120,22,0,5,36,13,...,0.327,0.344,0.670,84.0,157,14,3,9,1,0
24,150,665,587,96,164,27,3,5,60,37,...,0.347,0.361,0.708,89.0,212,17,5,16,1,0
26,122,415,380,46,91,20,0,9,37,10,...,0.296,0.363,0.659,81.0,138,7,2,3,1,1
27,129,486,443,47,97,20,4,23,78,1,...,0.282,0.438,0.720,91.0,194,6,4,0,3,3
34,141,551,464,63,137,33,4,19,82,3,...,0.389,0.506,0.895,142.0,235,8,3,3,8,9
35,91,309,286,31,73,17,3,7,39,14,...,0.289,0.409,0.698,89.0,117,8,2,4,4,0


,Salary
91,9000000
782,414000
512,600000
325,2050000
696,421500
567,452180
364,1500000
740,417400
660,425000
509,640000


In [226]:
regr = linear_model.LinearRegression()
regr.fit(train_x.values, train_y.values)
# salary_predict(testingYear)
print('Weights Bitch!!!! = \n', regr.coef_)
predicted = regr.predict(test_x.values)
# print test_y.values
test_y.Salary = test_y.Salary.astype(float)
diff = predicted - (test_y.values)
binarizer = preprocessing.Binarizer().fit(diff)
binarizer.transform(diff)

# overpaid = np.where((predicted - (test_y.values)) < -10000000)[0]
# for over in overpaid: 
#     display(test_x.iloc[over])
    


('Weights Bitch!!!! = \n', array([[   -63513.39577444,    719379.03801797,   -689855.10802702,
            46961.61708598,     45532.84065801,    -61561.48527793,
          -197496.31182229,    114374.21350732,     40154.97047888,
            21140.45007711,   -219209.53423722,   -687987.12148597,
           -23598.56443633, -10628140.13333799,  -7185322.27994649,
        -10548071.09900525,  20978735.90368399,    -21692.4515594 ,
           -67898.62780381,     94162.05097427,   -730389.75513604,
          -981027.53922491,   -601015.10778907,    171482.79961222]]))


array([[ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
      